In [25]:
import pandas as pd
import numpy as np
df = pd.read_csv("Train_B_Tree.csv")

In [26]:
class Node:
    def __repr__(self):
        return "<Node Object with attribute "+str(self.attr)+" and bound "+str(self.bound)+" and decision "+str(self.decision)+">"
    def __init__(self, *args, **kwargs):
        try:
            self.parent = kwargs['parent']
        except:
            self.parent = None
        try:
            self.left = kwargs['left']
        except:
            self.left = None
        try:
            self.right = kwargs['right']
        except:
            self.right = None
        try:
            self.decision = kwargs['decision']
        except:
            self.decision = None
        try:
            self.attr = kwargs['attr']
        except:
            self.attr = None
        try:
            self.bound = kwargs['bound']
        except:
            self.bound = None

In [27]:
def regressiontree(df):
    train=df.sample(frac=0.7) #random state is a seed value
    test=df.drop(train.index)
    test.reset_index(drop=True,inplace=True)
    list_attr = train.columns.to_numpy()[:-1].tolist()
    NodeList=[]
    root = Node()
    split(train, root, 'left', list_attr, NodeList)
    return (root,train,test)

In [28]:
def decide(root,test):
    l=test.shape[0]
    sqerr = 0
    for i in range(0,l):
        start = root.left
        while(not start.decision):
            if(test[start.attr][i]<=(start.bound)):
                 start = start.left
            elif(test[start.attr][i]>(start.bound)):
                 start = start.right
        e_val = start.decision - test['csMPa'][i]
        sqerr = sqerr + e_val**2
    sqerr=sqerr/l
    return sqerr

In [29]:
def LS_ERROR(train, list_attr):
    input_train = train[list_attr]
    if(len(input_train.drop_duplicates())<=1):
        return 0
    else:
        #t = type(train['csMPa'][0])
        m = train['csMPa'].mean()
        ERROR = 0
        #print(len(train))
        for i in train.index:
            t=type(i)
            ERROR = ERROR + (train['csMPa'][i] - m)**2
        return ERROR

In [30]:
def WEIGHTED_LS(left, right, list_attr):
    lls = LS_ERROR(left, list_attr)
    rls = LS_ERROR(right, list_attr)
    return ((lls*len(left) + rls*len(right))/(len(left)+len(right)))

In [31]:
def split(root_train, parent, side, list_attr, NodeList):
    a = Node(parent=parent)
    input_train = root_train[list_attr]
    if(LS_ERROR(root_train, list_attr) == 0):
        a.decision = root_train['csMPa'].mean()
        if(side == 'left'):
            parent.left = a
        else:
            parent.right = a
        NodeList.append(a)
        return
    MIN_VAR = np.inf
    attr=None
    bound=0
    left=pd.DataFrame(columns=['cement','slag','flyash','water','superplasticizer','coarseaggregate','fineaggregate','age'])
    right=pd.DataFrame(columns=['cement','slag','flyash','water','superplasticizer','coarseaggregate','fineaggregate','age'])
    for attribute in list_attr:
        valid_set = root_train[attribute].unique().tolist()
        valid_set.sort()
        l =len(valid_set)
        if(l>=2):
            for i in range(0,l-1):
                left_temp = root_train[root_train[attribute]<=valid_set[i]]
                right_temp = root_train[root_train[attribute]>valid_set[i]]
                v =WEIGHTED_LS(left_temp, right_temp, list_attr)
                if(v<MIN_VAR):
                    left = left_temp
                    right = right_temp
                    attr = attribute
                    bound = (valid_set[i]+valid_set[i+1])/2
                    MIN_VAR = v
    print(MIN_VAR, attr, bound)
    a.attr = attr
    a.bound = bound
    if(side == 'left'):
        parent.left = a
    else:
        parent.right = a
    NodeList.append(a)
    split(left, a, 'left', list_attr, NodeList)
    split(right, a, 'right', list_attr, NodeList)
    return

In [32]:
#print(df[df['age']<=3])
root=Node()
train=pd.DataFrame(columns=['cement','slag','flyash','water','superplasticizer','coarseaggregate','fineaggregate','age'])
test=pd.DataFrame(columns=['cement','slag','flyash','water','superplasticizer','coarseaggregate','fineaggregate','age'])
err=np.inf
if __name__ == "__main__":
    for i in range(0,10):
        (roottemp,traintemp,testtemp)=regressiontree(df)
        error=decide(roottemp,testtemp)
        if(error<err):
            root=roottemp
            train=traintemp
            test=testtemp
            err=error

83293.27399153952 water 178.05
33172.309478260875 cement 270.6
9406.979000000001 age 42.0
4327.225375555556 age 21.0
731.3034177777778 age 8.5
210.22468695652174 water 159.1
73.21260000000001 superplasticizer 11.600000000000001
15.22662 cement 170.2
2.5788249999999993 cement 213.45
0.0 cement 192.2
0.0 cement 216.9
2.4637999999999995 cement 232.5
0.1925333333333341 cement 249.3
0.0 cement 252.05
58.180257142857144 slag 47.650000000000006
15.146299999999997 coarseaggregate 1057.9
1.398614285714287 cement 213.05
0.4257600000000011 fineaggregate 903.7
0.0021333333333333373 cement 173.2
0.0 cement 196.95
0.0 cement 203.64999999999998
0.0 cement 222.65
1.5122999999999986 cement 190.5
0.0 cement 186.15
3.542499999999994 coarseaggregate 1006.25
0.0 cement 169.25
0.0 cement 170.60000000000002
260.6711090909091 cement 203.35
51.43028461538462 coarseaggregate 1031.4499999999998
4.323660000000003 cement 183.55
0.012033333333333946 cement 169.7
0.0 cement 166.0
0.0 water 167.89999999999998
14.7963

0.0 cement 208.05
0.7905333333333324 flyash 96.2
0.0 cement 236.9
29.983625000000004 cement 213.75
0.0 cement 195.85
0.2872333333333353 cement 218.10000000000002
0.05208000000000028 flyash 96.2
0.0 water 189.60000000000002
0.00013333333333332765 cement 267.7
0.0 cement 236.9
373.92840000000007 age 5.0
64.436208 cement 187.95
11.7104 slag 181.2
2.77184 cement 130.65
0.0 cement 105.15
1.3872 cement 163.0
0.0 cement 178.45
3.246059999999999 water 197.75
0.4641333333333338 cement 148.3
0.0 cement 136.3
0.0 cement 154.05
32.57874 fineaggregate 758.1
9.118757142857142 cement 220.85
2.5208333333333353 cement 197.75
0.0 cement 207.75
0.24435000000000062 flyash 118.25
0.014700000000000121 cement 270.65
0.0 cement 232.85
4.6758999999999995 fineaggregate 857.6
1.405799999999998 coarseaggregate 969.5
0.35363333333333297 cement 270.4
0.0 cement 216.9
0.0 cement 270.0
0.4107000000000009 cement 245.0
0.0 cement 254.5
145.7766384615385 fineaggregate 790.6
30.762000000000018 coarseaggregate 933.7
0.0 c

127.70485384615387 water 186.5
21.541657142857158 fineaggregate 751.15
2.980079999999999 cement 389.75
0.0 cement 382.25
0.0 water 185.85
0.0 cement 383.2
9.109983333333341 age 59.0
0.024049999999999093 cement 396.5
0.0008333333333332386 cement 440.0
0.0 coarseaggregate 936.1
0.0 coarseaggregate 940.3
1425.3419199999998 coarseaggregate 889.95
290.872947368421 cement 317.5
11.132649999999988 water 198.25
3.4772333333333294 cement 280.2
0.0 cement 270.25
0.0 cement 299.0
23.086218181818207 water 188.0
0.11999999999999772 age 42.0
0.0 cement 457.0
3.052925000000007 age 42.0
0.9734333333333381 water 205.5
0.198860000000001 slag 166.89999999999998
3.333333333332007e-05 cement 406.15
0.0 cement 321.15
0.0 cement 330.75
0.0 water 205.5
537.5063548387096 slag 136.95
210.0372526315789 slag 85.5
36.0018 fineaggregate 651.5
0.13229999999999809 cement 438.8
0.0 cement 380.75
3.5918400000000004 age 225.0
0.6347999999999958 age 135.0
0.0 age 59.0
0.0 cement 323.0
43.50159090909089 slag 109.55
4.2576

65.65161538461538 superplasticizer 2.5
1.2468800000000027 coarseaggregate 1026.5
0.004799999999999795 cement 254.4
0.0 cement 224.0
0.0 cement 229.5
1.6921999999999997 slag 110.75
0.14293333333333252 water 198.2
0.002559999999999891 cement 253.55
0.0 cement 258.6
0.0 cement 249.5
2579.4397627659573 superplasticizer 6.550000000000001
999.9056923076921 fineaggregate 758.1500000000001
248.27405454545462 cement 236.75
80.00880000000004 coarseaggregate 967.5999999999999
29.093340000000005 age 225.0
7.053333333333337 cement 196.75
0.0 age 135.0
0.0 age 317.5
5.791260000000009 water 193.6
0.0 cement 207.0
0.48000000000000226 slag 231.1
0.0 cement 185.5
96.3092916666667 cement 243.85
2.5362500000000052 age 225.0
0.0 age 135.0
0.0 age 317.5
25.167049999999993 slag 57.0
7.1124600000000076 age 78.0
0.0 water 190.7
0.13230000000000108 water 190.7
0.0 superplasticizer 6.1
0.1541333333333332 age 135.0
0.0 cement 258.1
429.4180733333333 fineaggregate 862.1
161.20468947368423 age 95.0
45.8051000000000

5442.741168253969 fineaggregate 724.3
2156.149812962963 cement 437.5
812.9931222222222 age 59.0
328.13870000000003 slag 8.8
0.0 cement 370.5
124.51790833333337 superplasticizer 3.0
28.478533333333356 cement 310.70000000000005
6.5416333333333325 cement 296.0
0.0 cement 304.65
3.9216333333333324 cement 324.3
0.0 cement 356.25
0.4517500000000006 slag 128.85
0.025349999999999505 cement 310.65
3.333333333332007e-05 cement 282.15
0.0 slag 116.9
0.0 cement 310.15
36.01370476190477 fineaggregate 632.0
5.005888888888888 age 317.5
0.6639999999999965 cement 403.75
0.013333333333332767 cement 356.25
0.0 age 135.0
0.03000000000000085 age 225.0
0.0 age 135.0
0.0 cement 356.25
10.284466666666665 age 225.0
1.160600000000007 cement 361.0
0.8533500000000043 age 135.0
0.537633333333336 cement 296.0
0.0 cement 323.0
0.0 age 135.0
1.153933333333329 cement 361.0
0.30920000000000103 age 317.5
0.0 cement 323.0
0.0 cement 323.0
0.0 age 317.5
494.98275555555546 water 190.5
32.42076666666665 cement 482.0
0.85333

...

32.322 slag 56.4
1.4419428571428579 fineaggregate 862.1
0.1398499999999997 cement 289.85
0.026133333333333546 cement 279.55
0.0 cement 260.95000000000005
0.1365333333333336 cement 264.45000000000005
0.0 cement 283.75
12.200833333333335 age 5.0
0.0 cement 281.75
131.09169583333332 slag 162.8
27.198206249999995 fineaggregate 776.65
7.4289333333333305 cement 295.9
3.075579999999996 superplasticizer 6.1
0.37904999999999933 flyash 59.15
0.0005333333333333106 cement 266.6
0.0 cement 295.4
12.68343 cement 265.0
2.733400000000001 water 188.9
0.16535000000000008 water 185.35
3.333333333333191e-05 cement 245.0
0.0 cement 252.25
0.0 cement 252.5
0.061025000000000204 age 5.0
0.0 cement 283.0
0.0 cement 282.0
44.436000000000014 age 5.0
0.0 cement 269.1
6.775433333333342 slag 186.95
0.058799999999999485 coarseaggregate 1019.5
0.0 cement 261.5
0.9185333333333334 cement 262.75
0.0 cement 289.1
645.9310675675677 superplasticizer 7.85
253.3893380952381 slag 6.8
56.14338571428572 superplasticizer 2.25
5.

453.79354482758623 coarseaggregate 1054.6
114.36567647058824 flyash 110.9
48.67109999999999 cement 243.6
9.266599999999988 superplasticizer 9.55
0.6347999999999958 age 78.0
0.0 cement 225.3
0.0 age 78.0
20.53512499999999 age 78.0
0.0 cement 263.1
0.0 cement 272.4
14.092575000000005 slag 84.4
0.0005333333333333106 flyash 170.25
0.0 cement 221.05
2.710759999999996 cement 166.2
0.0 age 78.0
1.2674999999999945 cement 170.60000000000002
0.0 age 78.0
85.54961666666667 cement 201.35
12.244083333333322 water 164.35
6.308149999999987 age 78.0
0.11603333333333188 cement 190.5
0.0 cement 179.60000000000002
0.0 cement 185.85000000000002
20.60343333333334 water 158.75
2.5291200000000034 slag 49.05
1.3067999999999957 age 78.0
0.0 cement 221.9
0.0 cement 213.75
31937.302559501557 water 183.9
13607.329854838712 age 21.0
1433.7090224489793 age 5.0
316.9373217391305 flyash 39.5
102.00933333333333 cement 389.25
23.38249999999998 coarseaggregate 943.7
0.013333333333333239 cement 321.25
0.0 cement 356.15
0

79409.22733495147 cement 277.15
27604.59836139411 age 21.0
2202.6605709401715 age 10.5
1006.2299541176469 cement 213.05
394.41355000000004 cement 173.4
119.44861739130438 coarseaggregate 988.8
27.048746153846157 age 5.0
6.8332999999999995 cement 149.15
2.1800599999999997 water 197.75
0.11999999999999987 cement 109.0
0.0 cement 124.5
0.0 cement 131.95
0.0 cement 165.0
8.333083333333336 slag 167.7
0.22504999999999964 cement 163.0
0.0005333333333333343 cement 130.65
0.0 cement 105.15
0.0 cement 124.5
29.610080000000004 superplasticizer 6.75
13.150214285714284 water 188.85
0.5180400000000008 coarseaggregate 1058.65
0.0002999999999999872 cement 162.45
0.0 cement 168.2
0.0 cement 151.85
0.0 age 5.0
0.11213333333333336 cement 166.0
0.0 cement 167.2
189.00674800000004 fineaggregate 754.65
73.83868888888888 cement 198.5
16.00702 fineaggregate 715.2
0.0 cement 187.7
0.1875 cement 175.9
0.0 cement 185.0
6.975650000000002 age 5.0
0.0 cement 207.75
0.0 cement 207.75
74.50680000000003 water 169.8999

23.765027272727274 cement 263.5
7.724066666666671 cement 206.05
0.7803000000000012 age 270.0
0.0 cement 199.3
2.8574000000000033 age 78.0
0.0 water 189.60000000000002
0.6565250000000036 cement 236.2
0.0 cement 217.25
0.0 water 189.60000000000002
0.0 age 270.0
42.03945454545456 cement 234.9
15.49591428571428 age 95.0
1.320619999999999 coarseaggregate 952.6
0.0 cement 171.05
0.1451999999999985 cement 206.95
0.0 cement 153.25
0.0 cement 231.9
3.1862499999999994 cement 245.0
0.0 water 193.5
0.0 age 227.5
974.6618825000002 cement 213.05
305.52507727272723 slag 21.1
130.15917857142853 cement 192.7
39.939333333333316 age 78.0
0.017633333333332852 flyash 125.30000000000001
0.0 cement 186.05
7.679999999999992 flyash 125.30000000000001
0.0 cement 186.05
21.81352500000002 fineaggregate 904.8499999999999
5.625314285714287 age 78.0
1.2870500000000018 flyash 112.65
0.23520000000000194 cement 212.55
0.0 cement 203.6
0.0021333333333332424 cement 203.35
0.0 cement 212.3
23.958150000000003 cement 168.15

76.96575555555555 age 118.0
14.29299999999997 cement 515.0
5.866349999999993 cement 498.0
1.7176333333333274 age 42.0
0.0 water 205.5
0.0 cement 526.5
8.978699999999991 cement 512.5
0.0 age 225.0
2340.679946376812 coarseaggregate 953.3
1016.2910071428574 slag 152.0
476.45024782608704 water 195.5
88.6151 fineaggregate 828.9000000000001
37.07548888888891 age 59.0
6.679100000000005 cement 293.5
0.0 slag 133.85
0.6554799999999975 cement 327.0
0.00013333333333337503 cement 374.45
0.0 coarseaggregate 940.3
0.0 coarseaggregate 940.3
13.653333333333327 age 59.0
0.0 cement 311.0
69.33750000000002 flyash 123.0
2.9356571428571487 flyash 117.9
0.19324000000000016 slag 69.95
3.3333333333320074e-05 cement 310.1
0.0 cement 298.1
0.1921999999999984 cement 326.25
0.0 cement 322.0
0.14164000000000115 cement 348.5
3.333333333332007e-05 cement 325.75
0.0 cement 330.75
507.273017073171 cement 328.0
88.77288125000003 age 59.0
11.457800000000002 cement 304.5
1.7235599999999995 water 185.5
0.0 cement 288.0
0.

In [35]:
train

,cement,slag,flyash,water,superplasticizer,coarseaggregate,fineaggregate,age,csMPa
0,331.0,0.0,0.0,192.0,0.0,1025.0,821.0,90,37.91
1,480.0,0.0,0.0,192.0,0.0,936.2,712.2,28,43.94
2,313.0,145.0,0.0,127.0,8.0,1000.0,822.0,28,44.52
3,250.0,0.0,95.7,187.4,5.5,956.9,861.2,3,13.82
4,139.6,209.4,0.0,192.0,0.0,1047.0,806.9,90,39.36
...,...,...,...,...,...,...,...,...,...
716,491.0,26.0,123.0,201.0,3.9,822.0,699.0,28,57.92
717,439.0,177.0,0.0,186.0,11.1,884.9,707.9,28,66.00
718,229.7,0.0,118.2,195.2,6.1,1028.1,757.6,56,31.35
719,231.8,0.0,121.6,174.0,6.7,1056.4,778.5,100,45.84


In [34]:
train.reset_index(drop=True,inplace=True)

In [36]:
train.to_csv('Train.csv')

In [37]:
test

,cement,slag,flyash,water,superplasticizer,coarseaggregate,fineaggregate,age,csMPa
0,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
1,266.0,114.0,0.0,228.0,0.0,932.0,670.0,90,47.03
2,266.0,114.0,0.0,228.0,0.0,932.0,670.0,28,45.85
3,427.5,47.5,0.0,228.0,0.0,932.0,594.0,270,43.01
4,304.0,76.0,0.0,228.0,0.0,932.0,670.0,28,47.81
...,...,...,...,...,...,...,...,...,...
304,321.4,0.0,127.9,182.5,11.5,870.1,779.7,28,37.27
305,298.2,0.0,107.0,209.7,11.1,879.6,744.2,28,31.88
306,159.8,250.0,0.0,168.4,12.2,1049.3,688.2,28,39.46
307,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28,44.28


In [38]:
test.to_csv('Test.csv')

In [40]:
print('Least square error for 10 random splits=',err)

Least square error for 10 random splits= 35.684428964401256
